## DMX interface

In [1]:
import wled_common_client, importlib    
importlib.reload(wled_common_client)
from wled_common_client import Wled, Wleds

In [2]:
import time

In [4]:
w = Wled("192.168.0.166")
w.cache_fs()
w.name = w.cfg["id"]["name"]
wleds = Wleds.from_one_node(w)

In [18]:
# wl1 = wleds["WLED-Dima-Office"]
wl1 = wleds["WLED-L-frame-2"]
wl1.cache_fs()

In [19]:
wl1.dmx.start()

In [20]:
data = [255, 255,0] * (wl1.dmx.n_leds//2 )
data += [0, 255,0] * (len(data) - wl1.dmx.n_leds)

In [21]:
for i in range(1000):
    wl1.dmx.set_data(data)
    time.sleep(0.01)
    for i in range(3):
        data.insert(0, data.pop())

In [16]:
wl1.dmx.set_data(data)

In [22]:
wl1.dmx.stop()

## DMX experiments

In [30]:
import sacn
import time
import math

In [36]:
# sender = sacn.sACNsender(bind_address="192.168.0.16")  # provide an IP-Address to bind to if you are using Windows and want to use multicast
sender = sacn.sACNsender()  # provide an IP-Address to bind to if you are using Windows and want to use multicast
sender.start()  # start the sending thread

In [42]:

sender.activate_output(1)  # start sending out data in the 1st universe
# sender.activate_output(2)  # start sending out data in the 1st universe
sender[1].multicast = True  # set multicast to True
sender[1].fps=10
# sender[2].multicast = True  # set multicast to True
# sender.destination = "192.168.0.14"  # or provide unicast information.
# sender[1].destination = "192.168.0.122"  # or provide unicast information.
# sender[2].destination = sender[1].destination  # or provide unicast information.
# sender[1].fps = 60
# Keep in mind that if multicast is on, unicast is not used



In [41]:
sender[1].dmx_data = tuple([0, 0, 0] * 100 + [255, 0, 0] * 70 )  # some test DMX data


In [40]:
sender[2].dmx_data = tuple([0, 0, 0]*10 + [255, 0, 0]*166)  # some test DMX data


AttributeError: 'NoneType' object has no attribute 'dmx_data'

In [35]:
sender.stop()  # do not forget to stop the sender

In [28]:
math.ceil(1000/ (512//3))

6

## Semi-manual updates

In [5]:
import wled_common_client, importlib    
importlib.reload(wled_common_client)
from wled_common_client import Wled, Wleds

In [6]:
from omegaconf import DictConfig, OmegaConf, ListConfig

In [9]:
w = Wled("192.168.0.109")
w.cache_fs()
w.name = w.cfg["id"]["name"]
wleds = Wleds.from_one_node(w)

In [25]:
for wled in wleds:
    wled.cache_fs()
    # wled.dump_fs("config_dump_names/{name}")

In [29]:
for wled in wleds:
    wled.update_time()
    print (wled.name)

WLED-L-frame-2
WLED-M-frame-1
WLED-spatial-test
WLED-L-frame-1


In [ ]:
for wled in wleds:
    wled.dump_omegaconf()

In [ ]:
for wled in wleds:
    wled.presets = OmegaConf.to_container(OmegaConf.load("default_presets.yaml"))
    wled.upload_presets()

In [ ]:
for wled in wleds:
    oc_dir = wled.get_fs_dump_dir("omegaconf_source/{name}")
    cfg_yaml = f"{oc_dir}/cfg.yaml"
    ww = Wled.from_omegaconf(additional_confs=[cfg_yaml])
    ww.ip = wled.ip
    ww.upload_cfg()

In [ ]:
for wled in wleds:
    oc_dir = self.get_fs_dump_dir(wled_common_client.OMEGACONF_DUMP_DIR)
    cfg_yaml = f"{oc_dir}/cfg.yaml"
    presets_yaml = f"{oc_dir}/presets.yaml"
    wled.presets = OmegaConf.load(presets_yaml)
    wled.cfg = OmegaConf.load(presets_yaml)

In [ ]:
yaml.load()

## Tidying up presets

In [158]:
import json
from ruamel.yaml import YAML
yaml = YAML()

In [119]:
with open("config_dump/WLED-Dima-Office/presets.json") as js:
    p = yaml.load(js)

In [120]:
# yaml.compact_seq_seq = True
# yaml.compact_seq_map = False
yaml.width = 10**7
yaml.top_level_colon_align = True


In [121]:
p.fa.set_block_style()
for k in p.keys():
    p[k].fa.set_block_style()

In [122]:
with open("default_presets.yaml", "w") as y:
    yaml.dump(p, y)


# MIDI

In [1]:
import sys
import os

import pygame as pg
import pygame.midi


def print_device_info():
    pygame.midi.init()
    _print_device_info()
    pygame.midi.quit()


def _print_device_info():
    for i in range(pygame.midi.get_count()):
        r = pygame.midi.get_device_info(i)
        (interf, name, input, output, opened) = r

        in_out = ""
        if input:
            in_out = "(input)"
        if output:
            in_out = "(output)"

        print(
            "%2i: interface :%s:, name :%s:, opened :%s:  %s"
            % (i, interf, name, opened, in_out)
        )


def input_main(device_id=None):
    pg.init()
    pg.fastevent.init()
    event_get = pg.fastevent.get
    event_post = pg.fastevent.post

    pygame.midi.init()

    _print_device_info()

    if device_id is None:
        input_id = pygame.midi.get_default_input_id()
    else:
        input_id = device_id

    print("using input_id :%s:" % input_id)
    i = pygame.midi.Input(input_id)

    pg.display.set_mode((1, 1))

    going = True
    while going:
        events = event_get()
        for e in events:
            if e.type in [pg.QUIT]:
                going = False
            if e.type in [pg.KEYDOWN]:
                going = False
            if e.type in [pygame.midi.MIDIIN]:
                print(e)

        if i.poll():
            midi_events = i.read(10)
            # convert them into pygame events.
            midi_evs = pygame.midi.midis2events(midi_events, i.device_id)

            for m_e in midi_evs:
                event_post(m_e)

    del i
    pygame.midi.quit()

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
input_main()